In [1]:
### useful: download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')



downloading chinook.zip ...........................................................................

In [2]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)



In [8]:
#OPEN database
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine("sqlite:///chinook.db", echo=True, future=True)
conn = engine.connect()

In [9]:
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()



2023-08-07 16:57:02,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 16:57:02,314 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-08-07 16:57:02,314 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-07 16:57:02,316 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-08-07 16:57:02,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-07 16:57:02,317 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("albums")
2023-08-07 16:57:02,318 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-07 16:57:02,319 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("artists")
2023-08-07 16:57:02,319 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-07 16:57:02,319 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("customers")
2023-08-07 16:57:02,320 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [12]:
#print tables names
print(metadata.tables.keys())

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'tracks', 'media_types', 'playlist_track', 'playlists'])


In [56]:
# print out the first three tracks in the tracks table SQL option
from sqlalchemy import text, select

stmt = text("SELECT * FROM tracks LIMIT 3")
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2023-08-07 17:05:40,938 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:05:40,938 INFO sqlalchemy.engine.Engine SELECT * FROM tracks LIMIT 3
2023-08-07 17:05:40,940 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()
[(1, 'For Those About To Rock (We Salute You)', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 343719, 11170334, 0.99), (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, 0.99), (3, 'Fast As a Shark', 3, 2, 1, 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619, 3990994, 0.99)]
2023-08-07 17:05:40,941 INFO sqlalchemy.engine.Engine ROLLBACK


In [65]:
# print out the first three tracks in the tracks table DBAPI option
from sqlalchemy import text, select , join
tracks = Base.classes['tracks']
stmt = select(tracks).limit(3)
# print(stmt)
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2023-08-07 17:12:26,811 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:12:26,813 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT ? OFFSET ?
2023-08-07 17:12:26,813 INFO sqlalchemy.engine.Engine [cached since 254.9s ago] (3, 0)
[(1, 'For Those About To Rock (We Salute You)', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 343719, 11170334, Decimal('0.99')), (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, Decimal('0.99')), (3, 'Fast As a Shark', 3, 2, 1, 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619, 3990994, Decimal('0.99'))]
2023-08-07 17:12:26,815 INFO sqlalchemy.engine.Engine ROLLBACK


In [68]:
#print out the track name and albums title of the first 20 tracks in the `tracks` table
albums = Base.classes['albums']
stmt = select(tracks.Name,albums).select_from(join(tracks,albums)).limit(20)
display_results(stmt)


2023-08-07 17:13:33,375 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:13:33,376 INFO sqlalchemy.engine.Engine SELECT tracks."Name", albums."AlbumId", albums."Title", albums."ArtistId" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId"
 LIMIT ? OFFSET ?
2023-08-07 17:13:33,377 INFO sqlalchemy.engine.Engine [generated in 0.00121s] (20, 0)
2023-08-07 17:13:33,382 INFO sqlalchemy.engine.Engine ROLLBACK


,Name,AlbumId,Title,ArtistId
0,For Those About To Rock (We Salute You),1,For Those About To Rock We Salute You,1
1,Put The Finger On You,1,For Those About To Rock We Salute You,1
2,Let's Get It Up,1,For Those About To Rock We Salute You,1
3,Inject The Venom,1,For Those About To Rock We Salute You,1
4,Snowballed,1,For Those About To Rock We Salute You,1
5,Evil Walks,1,For Those About To Rock We Salute You,1
6,C.O.D.,1,For Those About To Rock We Salute You,1
7,Breaking The Rules,1,For Those About To Rock We Salute You,1
8,Night Of The Long Knives,1,For Those About To Rock We Salute You,1
9,Spellbound,1,For Those About To Rock We Salute You,1



SELECT tracks."Name", albums."AlbumId", albums."Title", albums."ArtistId" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId"
 LIMIT :param_1



In [70]:
stmt = select(tracks.Name,albums.Title).join(albums).limit(20)
display_results(stmt)


2023-08-07 17:14:38,747 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:14:38,747 INFO sqlalchemy.engine.Engine SELECT tracks."Name", albums."Title" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId"
 LIMIT ? OFFSET ?
2023-08-07 17:14:38,748 INFO sqlalchemy.engine.Engine [generated in 0.00131s] (20, 0)
2023-08-07 17:14:38,750 INFO sqlalchemy.engine.Engine ROLLBACK


,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You



SELECT tracks."Name", albums."Title" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId"
 LIMIT :param_1



In [74]:
#print out the first 10 track sales from the invoice_items table
invoice = Base.classes['invoice_items']
stmt = select(invoice).limit(10)
display_results(stmt)

2023-08-07 17:17:50,670 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:17:50,671 INFO sqlalchemy.engine.Engine SELECT invoice_items."InvoiceLineId", invoice_items."InvoiceId", invoice_items."TrackId", invoice_items."UnitPrice", invoice_items."Quantity" 
FROM invoice_items
 LIMIT ? OFFSET ?
2023-08-07 17:17:50,672 INFO sqlalchemy.engine.Engine [cached since 75.52s ago] (10, 0)
2023-08-07 17:17:50,674 INFO sqlalchemy.engine.Engine ROLLBACK


,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
5,6,2,12,0.99,1
6,7,3,16,0.99,1
7,8,3,20,0.99,1
8,9,3,24,0.99,1
9,10,3,28,0.99,1



SELECT invoice_items."InvoiceLineId", invoice_items."InvoiceId", invoice_items."TrackId", invoice_items."UnitPrice", invoice_items."Quantity" 
FROM invoice_items
 LIMIT :param_1



In [75]:
stmt = select(tracks.Name,invoice.Quantity).join(tracks).limit(10)
display_results(stmt)

2023-08-07 17:18:02,853 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:18:02,853 INFO sqlalchemy.engine.Engine SELECT tracks."Name", invoice_items."Quantity" 
FROM invoice_items JOIN tracks ON tracks."TrackId" = invoice_items."TrackId"
 LIMIT ? OFFSET ?
2023-08-07 17:18:02,854 INFO sqlalchemy.engine.Engine [generated in 0.00116s] (10, 0)
2023-08-07 17:18:02,856 INFO sqlalchemy.engine.Engine ROLLBACK


,Name,Quantity
0,Balls to the Wall,1
1,Restless and Wild,1
2,Put The Finger On You,1
3,Inject The Venom,1
4,Evil Walks,1
5,Breaking The Rules,1
6,Dog Eat Dog,1
7,Overdose,1
8,Love In An Elevator,1
9,Janie's Got A Gun,1



SELECT tracks."Name", invoice_items."Quantity" 
FROM invoice_items JOIN tracks ON tracks."TrackId" = invoice_items."TrackId"
 LIMIT :param_1



In [87]:
# print the names of top 10 tracks sold, and how many they times they were sold
from sqlalchemy import func,desc,asc
stmt = select(tracks.Name,func.sum(invoice.Quantity).label("sold")).join(invoice).group_by(tracks.TrackId).order_by(desc("sold")).limit(10)
display_results(stmt)

2023-08-07 17:28:03,029 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:28:03,030 INFO sqlalchemy.engine.Engine SELECT tracks."Name", sum(invoice_items."Quantity") AS sold 
FROM tracks JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" GROUP BY tracks."TrackId" ORDER BY sold DESC
 LIMIT ? OFFSET ?
2023-08-07 17:28:03,031 INFO sqlalchemy.engine.Engine [generated in 0.00131s] (10, 0)
2023-08-07 17:28:03,035 INFO sqlalchemy.engine.Engine ROLLBACK


,Name,sold
0,Balls to the Wall,2
1,Inject The Venom,2
2,Snowballed,2
3,Overdose,2
4,Deuces Are Wild,2
5,Not The Doctor,2
6,Por Causa De Você,2
7,Welcome Home (Sanitarium),2
8,Snowblind,2
9,Cornucopia,2



SELECT tracks."Name", sum(invoice_items."Quantity") AS sold 
FROM tracks JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" GROUP BY tracks."TrackId" ORDER BY sold DESC
 LIMIT :param_1



In [98]:
# Who are the top 10 highest selling artists?
artists = Base.classes['artists']
stmt = select(func.sum(invoice.Quantity).label("sold"), artists.Name).join(tracks).join(albums).join(artists).group_by(artists.ArtistId).order_by(desc("sold")).limit(10)
display_results(stmt)

2023-08-07 17:35:17,315 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-07 17:35:17,316 INFO sqlalchemy.engine.Engine SELECT sum(invoice_items."Quantity") AS sold, artists."Name" 
FROM invoice_items JOIN tracks ON tracks."TrackId" = invoice_items."TrackId" JOIN albums ON albums."AlbumId" = tracks."AlbumId" JOIN artists ON artists."ArtistId" = albums."ArtistId" GROUP BY artists."ArtistId" ORDER BY sold DESC
 LIMIT ? OFFSET ?
2023-08-07 17:35:17,317 INFO sqlalchemy.engine.Engine [generated in 0.00137s] (10, 0)
2023-08-07 17:35:17,321 INFO sqlalchemy.engine.Engine ROLLBACK


,sold,Name
0,140,Iron Maiden
1,107,U2
2,91,Metallica
3,87,Led Zeppelin
4,45,Os Paralamas Do Sucesso
5,44,Deep Purple
6,42,Faith No More
7,41,Lost
8,40,Eric Clapton
9,39,R.E.M.



SELECT sum(invoice_items."Quantity") AS sold, artists."Name" 
FROM invoice_items JOIN tracks ON tracks."TrackId" = invoice_items."TrackId" JOIN albums ON albums."AlbumId" = tracks."AlbumId" JOIN artists ON artists."ArtistId" = albums."ArtistId" GROUP BY artists."ArtistId" ORDER BY sold DESC
 LIMIT :param_1

